In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

# Change the plot size.
plt.rcParams['figure.figsize'] = [18.0, 12.0]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pandas_summary import DataFrameSummary

In [ ]:
# import my Finance Analysis Tools (fat)
import finance as fat

In [ ]:
# full imports of fastai fail on windows, so just import some of it for now
from fastai.structured import add_datepart, apply_cats, proc_df

Don't run this on windows, only on paperspace...

In [ ]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [ ]:
PATH = 'data/gold-test'

## Load basic data

In [ ]:
ticker = 'IAU'
data = fat.get_price_data(ticker)

In [ ]:
data.tail()

## Create Features

We're going to try to be predicting tomorrow's closing price, so add it as a column.

In [ ]:
data['Tomorrow Adj Close'] = data['Adj Close'].shift(-1)

In [ ]:
data.tail()

We'll also add google trends for searches for  "gold price usd"

In [ ]:
search = "gold price usd"
trends = fat.get_google_trends_df(data, search)

In [ ]:
trends.tail()

In [ ]:
#related_queries

In [ ]:
data = pd.DataFrame.join(data, trends)

In [ ]:
data = data.dropna()
data[-14:]

In [ ]:
plot_data = data[['Adj Close', search]]
plot_data.plot(secondary_y = [search])

Add separate columns for various date parts.

In [ ]:
data = data.reset_index()
add_datepart(data, 'Date', drop=False)
data = data.set_index('Date')

In [ ]:
data.tail()

Add in columns for several Simple Moving Averages for various periods of days.

In [ ]:
sma_pct_diff = fat.get_sma_pct_diff_df(data, 'Adj Close')
del sma_pct_diff['Adj Close']
data = pd.DataFrame.join(data, sma_pct_diff)

In [ ]:
data = data.dropna()

In [ ]:
data.tail()

In [ ]:
DataFrameSummary(data).summary().T

In [ ]:
data.columns

Break the columns up into "category variables" and "continuous variables".

In [ ]:
cat_vars = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Days since updated gold price usd',
            'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

contin_vars = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'gold price usd', 'Elapsed',
               'pct diff Adj Close SMA3', 'pct diff Adj Close SMA5', 'pct diff Adj Close SMA10', 'pct diff Adj Close SMA20', 
               'pct diff Adj Close SMA50', 'pct diff Adj Close SMA100', 'pct diff Adj Close SMA200']

In [ ]:
print(f'len(cat_vars) = {len(cat_vars)}')
print(f'len(contin_vars) = {len(contin_vars)}')
print(f'len(data.columns) = {len(data.columns)}')
assert(len(data.columns) == (len(cat_vars) + len(contin_vars) + 1))

Split data into train and test sets.

In [ ]:
data_len = len(data)
train_len = int(data_len * .8)
joined = data[:train_len].copy()
joined_test = data[train_len:].copy()
#del joined_test['Tomorrow Adj Close']

In [ ]:
#train.tail().T

In [ ]:
#test.head().T

In [ ]:
#test.tail().T

In [ ]:
dep = 'Tomorrow Adj Close'
joined = joined.reset_index()
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

In [ ]:
joined.tail().T

In [ ]:
joined_test[dep] = 0
joined_test = joined_test.reset_index()
joined_test.head().T

In [ ]:
joined_test = joined_test[cat_vars+contin_vars+[dep,'Date']].copy()

In [ ]:
joined = joined.dropna()
joined_test = joined_test.dropna()

In [ ]:
for v in cat_vars: 
    joined[v] = joined[v].astype('category').cat.as_ordered()
    #joined_test[v] = joined_test[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(joined_test, joined)

We're going to run on full sample size.

In [ ]:
samp_size = len(joined)
joined_samp = joined.set_index("Date")

We can now process our data...

In [ ]:
df, y, nas, mapper = proc_df(joined_samp, dep, do_scale=True)
yl = np.log(y)

In [ ]:
df.head(2).T

In time series data, cross-validation is not random. Instead, our holdout data is generally the most recent data, as it would be in real application. This issue is discussed in detail in [this post](http://www.fast.ai/2017/11/13/validation-sets/) on our web site.

One approach is to take the last 25% of rows (sorted by date) as our validation set.

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [ ]:
#del joined_test[dep]

In [ ]:
#for v in cat_vars: 
#    #joined[v] = joined[v].astype('category').cat.as_ordered()
#    joined_test[v] = joined_test[v].astype('category').cat.as_ordered()

In [ ]:
joined_test = joined_test.set_index('Date')

In [ ]:
joined_test.tail().T

In [ ]:
df_test, _, nas, mapper = proc_df(joined_test, dep, do_scale=True,
                                  mapper=mapper, na_dict=nas)

In [ ]:
joined = joined.dropna()
joined_test = joined_test.dropna()

## Deep Learning

We're ready to put together our models.

Root-mean-squared percent error is the metric Kaggle used for this competition.

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

We can create a ModelData object directly from out data frame.

In [ ]:
# before: bs=128, changed to bs=8 to get results while finding learning rate
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=8,
                                       test_df=joined_test)

Create categorical variables.

In [ ]:
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]
cat_sz

We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
emb_szs

Find learning rate.

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.lr_find(1e-7,1e-2)

In [ ]:
m.sched.plot_lr()

In [ ]:
m.sched.plot(100)

### Sample

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 5, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.fit(lr, 2, metrics=[exp_rmspe], cycle_len=4)

In [ ]:
m.save('val0')

In [ ]:
m.load('val0')

In [ ]:
x,y=m.predict_with_targs()

In [ ]:
exp_rmspe(x,y)

In [ ]:
pred_test=m.predict(is_test=True)

In [ ]:
pred_test = np.exp(pred_test)

In [ ]:
joined_test[dep]=pred_test

In [ ]:
csv_fn=f'{PATH}tmp/sub.csv'

In [ ]:
joined_test[[dep]].to_csv(csv_fn, index=False)